# ⚛️ H₂ VQE — Noise Scans, ΔE Shifts & Fidelity Analysis

This notebook explores how **noise** affects the **Variational Quantum Eigensolver (VQE)**
for the hydrogen molecule **H₂**, using a clean two-part educational + production structure.

---

# **🧩 Part 1 — Manual Noisy VQE (Educational)**

In this section we:

- Build a **noisy PennyLane QNode** by hand  
- Use a simple **TwoQubit-RY-CNOT ansatz**  
- Inject explicit:
  - `DepolarizingChannel(p)`
  - `AmplitudeDamping(p)`  
- Run a minimal optimisation loop with **GradientDescent**

This section illustrates *how noise enters a quantum circuit directly*.

---

# **🟦 Part 2 — Package-Level Noise Sweeps (Production Workflow)**

Using your VQE engine (`vqe.core`) we:

- Compute a **noiseless reference state**  
- Sweep noise levels using:
  - `run_vqe_noise_sweep`
  - `run_vqe_multi_seed_noise`
- Obtain:
  - **Energy shift ΔE ← E_noisy − E_reference**
  - **Fidelity vs. reference**
  - **Mean ± std** across seeds
- Automatically generate and save:
  - ΔE vs noise  
  - Fidelity vs noise  
  - Convergence curves  

This section represents the **real scientific workflow**, fully cached and reproducible.

---

In [ ]:
# Core scientific stack
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem
import matplotlib.pyplot as plt

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

# Package API
from vqe.core import (
    run_vqe,
    run_vqe_noise_sweep,
    run_vqe_multi_seed_noise,
)

from vqe_qpe_common.plotting import IMG_DIR

# Basic H₂ setup for the manual demo
symbols = ["H", "H"]
coordinates = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 0.7414],
])
basis = "sto-3g"

# Active space restriction for H₂ (2 electrons, 2 orbitals)
H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=0,
    basis=basis,
    active_electrons=2,
    active_orbitals=2,
)

num_wires = qubits
print(f"H₂ with basis={basis}: {num_wires} qubits")

# 🧩 Part 1 — Manual Noisy VQE Demo  

This section demonstrates **explicit noise injection** inside a PennyLane QNode.

We:

- Build a **TwoQubit-RY-CNOT** ansatz  
- Add noise channels manually  
- Run a small VQE loop  

This is purely for *educational insight* — the package API handles all of this internally.

In [ ]:
# Simple two-qubit ansatz: RY on qubit 0 → CNOT(0→1) → RY on qubit 1
def two_qubit_ry_cnot(params, wires):
    qml.RY(params[0], wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[1], wires=wires[1])

# Noise parameters for manual demo
p_dep_demo = 0.02
p_amp_demo = 0.04

dev_noisy = qml.device("default.mixed", wires=num_wires)

@qml.qnode(dev_noisy, diff_method="finite-diff")
def noisy_energy(params):
    # Apply ansatz
    two_qubit_ry_cnot(params, wires=[0, 1])

    # Inject noise per qubit
    for w in range(num_wires):
        qml.DepolarizingChannel(p_dep_demo, wires=w)
        qml.AmplitudeDamping(p_amp_demo, wires=w)

    return qml.expval(H)

# Manual VQE loop
opt = qml.GradientDescentOptimizer(stepsize=0.1)
params = 0.1 * np.random.randn(2, requires_grad=True)

energies_demo = []
n_steps_demo = 30

for step in range(n_steps_demo):
    params, e_prev = opt.step_and_cost(noisy_energy, params)
    e_val = float(noisy_energy(params))
    energies_demo.append(e_val)
    print(f"[demo] Step {step+1:02d}/{n_steps_demo} — E = {e_val:.6f} Ha")

plt.figure(figsize=(7, 4))
plt.plot(range(1, n_steps_demo + 1), energies_demo, marker="o")
plt.xlabel("Iteration")
plt.ylabel("Energy (Ha)")
plt.title("Manual Noisy VQE Demo — TwoQubit-RY-CNOT")
plt.grid(True, alpha=0.4)
plt.tight_layout()

fname_demo = os.path.join(IMG_DIR, "H2_Manual_Noisy_VQE_Demo.png")
plt.savefig(fname_demo, dpi=300)
plt.show()

fname_demo

# 🟦 Part 2 — Noiseless Reference via Package API

Before scanning noise levels, we compute a **noiseless reference state** using:

```python
run_vqe(...)
```

This returns:

- Final energy  
- Convergence trace  
- Statevector (`final_state_real`, `final_state_imag`)  

This state is used for **fidelity** and **ΔE shifts**.

In [ ]:
ref_result = run_vqe(
    molecule="H2",
    ansatz_name="RY-CZ",
    optimizer_name="Adam",
    n_steps=50,
    stepsize=0.2,
    noisy=False,
    plot=True,
    seed=0,
    force=False,
)

reference_energy = ref_result["energy"]
psi_real = np.array(ref_result["final_state_real"])
psi_imag = np.array(ref_result["final_state_imag"])
reference_state = psi_real + 1j * psi_imag

reference_energy

# 🔬 Part 3 — Single-Seed Noise Sweeps  

Using:

```python
run_vqe_noise_sweep(...)
```

we sweep noise levels and compute:

- Energy shift ΔE  
- Fidelity vs reference  

The function automatically:

- Runs VQE with each noise level  
- Computes reference internally (unless provided)  
- Generates a 2-panel figure  
- Saves plots to `images/`  

In [ ]:
# Sweep depolarizing noise
run_vqe_noise_sweep(
    molecule="H2",
    ansatz_name="RY-CZ",
    optimizer_name="Adam",
    steps=30,
    depolarizing_probs=np.arange(0.0, 0.11, 0.02),
    amplitude_damping_probs=np.zeros(6),
    force=False,
    mapping="jordan_wigner",
)

In [ ]:
# Sweep amplitude damping noise
run_vqe_noise_sweep(
    molecule="H2",
    ansatz_name="RY-CZ",
    optimizer_name="Adam",
    steps=30,
    depolarizing_probs=np.zeros(6),
    amplitude_damping_probs=np.arange(0.0, 0.11, 0.02),
    force=False,
    mapping="jordan_wigner",
)

# 📈 Part 4 — Multi-Seed Noise Statistics  

To understand **run-to-run variability**, we call:

```python
run_vqe_multi_seed_noise(...)
```

For each noise level:

- VQE runs across multiple seeds  
- ΔE and fidelity are aggregated  
- Mean ± std is computed  

The resulting figures show:

- ΔE vs noise (error bars)  
- Fidelity vs noise (error bars)  

All plots are saved automatically.

In [ ]:
# Multi-seed depolarizing noise
run_vqe_multi_seed_noise(
    molecule="H2",
    ansatz_name="RY-CZ",
    optimizer_name="Adam",
    steps=30,
    stepsize=0.2,
    seeds=np.arange(0, 5),
    noise_type="depolarizing",
    depolarizing_probs=np.arange(0.0, 0.11, 0.02),
    force=False,
    mapping="jordan_wigner",
)

In [ ]:
# Multi-seed amplitude-damping noise
run_vqe_multi_seed_noise(
    molecule="H2",
    ansatz_name="RY-CZ",
    optimizer_name="Adam",
    steps=30,
    stepsize=0.2,
    seeds=np.arange(0, 5),
    noise_type="amplitude",
    depolarizing_probs=np.arange(0.0, 0.11, 0.02),   # reused internally
    force=False,
    mapping="jordan_wigner",
)

# 🧾 Summary

In this notebook you:

### ✔ Built a manual noisy VQE to see how noise is injected  
### ✔ Computed a noiseless reference state with `run_vqe`  
### ✔ Performed single-seed noise sweeps (ΔE vs noise, fidelity vs noise)  
### ✔ Performed multi-seed noise statistics for robust analysis  

This completes a **full educational + production-ready** exploration of VQE
under noise for H₂.